![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/work-with-data/dataprep/how-to-guides/subsetting-sampling.png)

# Sampling and Subsetting


Once a Dataflow has been created, it is possible to act on only a subset of the records contained in it. This can help when working with very large datasets or when only a portion of the records is truly relevant.

## Head

The `head` method will take the number of records specified, run them through the transformations in the Dataflow, and then return the result as a Pandas dataframe.

In [ ]:
import azureml.dataprep as dprep

dflow = dprep.read_csv('../data/crime_duplicate_headers.csv')
dflow.head(5)

## Take

The `take` method adds a step to the Dataflow that will keep the number of records specified (counting from the beginning) and drop the rest. Unlike `head`, which does not modify the Dataflow, all operations applied on a Dataflow on which `take` has been applied will affect only the records kept.

In [ ]:
dflow_top_five = dflow.take(5)
dflow_top_five.to_pandas_dataframe()

## Skip

It is also possible to skip a certain number of records in a Dataflow, such that transformations are only applied after a specific point. Depending on the underlying data source, a Dataflow with a `skip` step might still have to scan through the data in order to skip past the records.

In [ ]:
dflow_skip_top_one = dflow_top_five.skip(1)
dflow_skip_top_one.to_pandas_dataframe()

## Take Sample

In addition to taking records from the top of the dataset, it's also possible to take a random sample of the dataset. This is done through the `take_sample(probability, seed=None)` method. This method will scan through all of the records available in the Dataflow and include them based on the probability specified. The `seed` parameter is optional. If a seed is not provided, a stable one is generated, ensuring that the results for a specific Dataflow remain consistent. Different calls to `take_sample` will receive different seeds.

In [ ]:
dflow_sampled = dflow.take_sample(0.1)
dflow_sampled.to_pandas_dataframe()

`skip`, `take`, and `take_sample` can all be combined. With this, we can achieve behaviors like getting a random 10% sample fo the middle N records of a dataset.

In [ ]:
seed = 1
dflow_nested_sample = dflow.skip(1).take(5).take_sample(0.5, seed)
dflow_nested_sample.to_pandas_dataframe()

## Take Stratified Sample
Besides sampling all by a probability, we also have stratified sampling, provided the strata and strata weights, the probability to sample each stratum with.
This is done through the `take_stratified_sample(columns, fractions, seed=None)` method.
For all records, we will group each record by the columns specified to stratify, and based on the stratum x weight information in `fractions`, include said record.

Seed behavior is same as in `take_sample`.

If a stratum is not specified or the record cannot be grouped by said stratum, we default the weight to sample by to 0 (it will not be included).

The order of `fractions` must match the order of `columns`.

In [ ]:
fractions = {}
fractions[('ASSAULT',)] = 0.5
fractions[('BATTERY',)] = 0.2
fractions[('ARSON',)] = 0.5
fractions[('THEFT',)] = 1.0

columns = ['Primary Type']

single_strata_sample = dflow.take_stratified_sample(columns=columns, fractions = fractions, seed = 42)
single_strata_sample.head(5)

Stratified sampling on multiple columns is also supported.

In [ ]:
fractions = {}
fractions[('ASSAULT', '560')] = 0.5
fractions[('BATTERY', '460')] = 0.2
fractions[('ARSON', '1020')] = 0.5
fractions[('THEFT', '820')] = 1.0

columns = ['Primary Type', 'IUCR']

multi_strata_sample = dflow.take_stratified_sample(columns=columns, fractions = fractions, seed = 42)
multi_strata_sample.head(5)

## Caching
It is usually a good idea to cache the sampled Dataflow for later uses.

See [here](cache.ipynb) for more details about caching.